In [1]:
# the import statements:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import timedelta
from matplotlib.ticker import ScalarFormatter
import machine_learning_protocols as ml
import sklearn as sk
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsRegressor 
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler

In [2]:
def clean_donki(file_path):
    """
    Clean the DONKI dataset and return a DataFrame with selected columns.
    
    Notes
    -----
    The expected format for 'time_21_5' is '%m/%d/%y %H:%M'.
    """

    # Error handling in case the file is not there
    try:
        clean_data = pd.read_excel(file_path)
    except FileNotFoundError:
        raise FileNotFoundError(f"The file at {file_path} does not exist.")

    # get only the data listed above:
    clean_data = clean_data[["long", "lat", "speed", "half_width", "time_21_5", "file_name", "cme_transit_obs", "cme_transit_sim", "PE"]]

    # convert the time_21_5 into a datetime object:
    clean_data['time_21_5'] = pd.to_datetime(clean_data['time_21_5'], format='%m/%d/%y %H:%M')

    # return the resulting data set. This is the cleaned donki data set
    return clean_data

In [3]:
def clean_omni_data(file_path):
    """
    Cleans OMNI solar wind data from a specified file.
    
    Parameters:
    - file_path (str): Path to the data file.
    
    Returns:
    - DataFrame: Cleaned and processed OMNI data, with EPOCH_Time converted to datetime,
      bad values replaced, magnetic field magnitude calculated and converted to Gauss.
    
    Note:
    - Assumes bad values are represented as -1.0000000000000001e+31.
    - EPOCH_Time format should match '%d-%m-%YT%H:%M:%S.%f'.
    """

    # Load the data
    sw_ob = pd.read_csv(file_path, delim_whitespace=True)
    
    # join the EPOCH date and time together and restore the index 
    sw_ob["EPOCH_Time"] = sw_ob.index + 'T' + sw_ob["EPOCH_Time"]
    sw_ob['EPOCH_Time'] = pd.to_datetime(sw_ob['EPOCH_Time'], format='%d-%m-%YT%H:%M:%S.%f')
    sw_ob = sw_ob.reset_index(drop=True)

    # Convert EPOCH_Time to datetime, assuming EPOCH_Time is correctly formatted
    # This might need adjustment based on the actual format in your data
    sw_ob['EPOCH_Time'] = pd.to_datetime(sw_ob['EPOCH_Time'], format='%d-%m-%Y %H:%M:%S.%f')

    # Cleaning bad values
    sw_ob.replace(-1.0000000000000001e+31, np.nan, inplace=True)

    # Select only the numeric columns for interpolation
    numeric_cols = sw_ob.select_dtypes(include=[np.number]).columns

    # Interpolate missing values only in numeric columns
    sw_ob[numeric_cols] = sw_ob[numeric_cols].interpolate(method='linear')

    # Calculate magnitude of the magnetic field
    sw_ob["B_mag(nT)"] = np.sqrt(sw_ob["BR_(RTN)(nT)"]**2 + sw_ob["BT_(RTN)(nT)"]**2 + sw_ob["BN_(RTN)(nT)"]**2)
    sw_ob.drop(columns=["BR_(RTN)(nT)", "BT_(RTN)(nT)", "BN_(RTN)(nT)"], inplace=True)

    # Convert magnetic field magnitude from nano Tesla to Gauss
    sw_ob["B_mag(G)"] = sw_ob["B_mag(nT)"]*1e-5
    sw_ob.drop(columns=["B_mag(nT)"], inplace=True)

    # Reset index
    sw_ob.reset_index(drop=True, inplace=True)

    return sw_ob

In [4]:
# Function for processing observed parameters:
def process_observed_parameters(sw_ob, cme_data):
    """
    Processes observed solar wind parameters relative to specific times of interest from donki CME data.
    
    Parameters: 
    sw_ob (pandas.DataFrame): 
        Dataframe containing observed solar wind data with columns: EPOCH_Time, BULK_FLOW_SPEED(km/s), 
        ION_DENSITY(N/cm3), TEMPERATURE(Deg_K), and B_mag(G).
        
    cme_data (pandas.DataFrame): 
        Dataframe containing the DONKI CME catalog data, which includes the "time_21_5" column that 
        specifies the time for which solar wind data should be fetched.
        
    Returns:
    new_df (pandas.DataFrame): 
        A dataframe that contains the solar wind data for the chosen time interval from each time_21_5 value.
    """
    # Boltzmann's constant
    k = 1.380649e-16

    # These are the rows to grab SW data 
    all_rows = []

    # counter 
    counter = 0

    for _, cme_row in cme_data.iterrows():
        # Get the time of interest from the CME data
        time = cme_row["time_21_5"]

        # Determine the dynamic time_range based on the rounded minimum of observed and simulated transit times
        time_range = round(min(cme_row["cme_transit_obs"], cme_row["cme_transit_sim"]))
        
        # Find the closest time in the observed solar wind data
        time_differences = (sw_ob['EPOCH_Time'] - time).abs()
        closest_time_idx = time_differences.idxmin()
        closest_time = sw_ob.loc[closest_time_idx, 'EPOCH_Time']

        # Define the range of times to extract data from
        end_time = closest_time + pd.Timedelta(hours=time_range)

        # Select the rows within the time range
        range_rows = sw_ob[(sw_ob['EPOCH_Time'] >= closest_time) & (sw_ob['EPOCH_Time'] < end_time)].copy()

        # Now, for each row in the range, create a new row with the time and parameters
        for _, row in range_rows.iterrows():
            new_row = {
                "time_21_5": time,  # this time is from cme_data, kept constant for all rows in this range
                "real_time_value": row['EPOCH_Time'],  # add the exact observation time
                "B_ob": row["B_mag(G)"],
                "V_ob": row["BULK_FLOW_SPEED(km/s)"],
                "n_ob": row["ION_DENSITY(N/cm3)"],
                "T_ob": row["TEMPERATURE(Deg_K)"]
            }
            all_rows.append(new_row)

        counter += 1

    # Create a new dataframe from the list of rows
    new_df = pd.DataFrame(all_rows)

    # calcualte and add the (total pressure = n*k*T + B^2/8*pi) exerted by observed solar wind
    new_df["total_pressure_ob"] = new_df["n_ob"]*k*new_df["T_ob"] + (new_df["B_ob"]**2)/(8*np.pi)

    return new_df

In [5]:
def process_simulated_parameters(cme_data, data_directory):
    """
    Processes and retrieves simulated solar wind parameters for given CME events based on the minimum of observed
    and simulated CME transit times, rounded to the nearest whole hour.

    Parameters:
    - cme_data (DataFrame): A DataFrame containing data on CME events, including their times, observed transit times,
      and simulated transit times.
    - data_directory (str): The path to the directory containing the simulation files.

    Returns:
    - DataFrame: A DataFrame containing the simulated solar wind parameters for the given CME events.
    """
    
    # Boltzman constant (in CGS units)
    k = 1.380649e-16
    
    all_rows = []
    
    for _, row in cme_data.iterrows():
        
        # Determine the dynamic time_range based on the rounded minimum of observed and simulated transit times
        time_range = round(min(row['cme_transit_obs'], row['cme_transit_sim']))

        # Retrieve and clean the file name
        file_name = row['file_name'].replace(" ", "")
        folder_name = f"{row['time_21_5'].year}-{row['time_21_5'].month:02d}"
        
        # Construct the full path to the simulation file
        file_name_full = os.path.join(data_directory, folder_name, str(file_name) + "_ENLIL_time_line.dat")

        if os.path.exists(file_name_full):
            # Read in the simulation data
            sim_data = pd.read_csv(file_name_full, delim_whitespace=True, header=None)
            sim_data.columns = sim_data.iloc[0]
            sim_data = sim_data.drop([0, 1]).reset_index(drop=True)

            # Ensure numeric columns are read correctly
            for col in ['B_enl', 'V_enl', 'n_enl', 'T_enl']:
                sim_data[col] = pd.to_numeric(sim_data[col], errors='coerce')

            # Create the timestamp for each data point
            sim_data['timestamp'] = pd.to_datetime(sim_data[['year', 'month', 'day', 'hour', 'minute']])
            sim_data = sim_data.drop(["year", "month", "day", "hour", "minute"], axis=1)

            # Find the closest time to 'time_21_5'
            closest_time_idx = (sim_data['timestamp'] - row['time_21_5']).abs().idxmin()
            closest_time = sim_data.loc[closest_time_idx, 'timestamp']

            # Initialize the list to collect data points
            hourly_data_points = []

            for h in range(time_range):  # Use dynamic, rounded time_range
                target_time = closest_time + timedelta(hours=h)
                # Check if this target time is within the simulation data's range
                if target_time <= sim_data['timestamp'].iloc[-1]:
                    # Find the index of the row with the timestamp closest to the target_time
                    closest_hourly_idx = (sim_data['timestamp'] - target_time).abs().idxmin()
                    closest_hourly_time = sim_data.loc[closest_hourly_idx, 'timestamp']

                    new_row = {
                        "time_21_5": row['time_21_5'],
                        "sim_time_value": closest_hourly_time,
                        "B_sim": sim_data.loc[closest_hourly_idx, "B_enl"] * 1e-5,  # Convert from nT to Gauss
                        "V_sim": sim_data.loc[closest_hourly_idx, "V_enl"],
                        "n_sim": sim_data.loc[closest_hourly_idx, "n_enl"],
                        "T_sim": sim_data.loc[closest_hourly_idx, "T_enl"] * 1000  # Convert from kilo Kelvin to Kelvin
                    }
                    hourly_data_points.append(new_row)

            all_rows.extend(hourly_data_points)

    # Create a new DataFrame from the list of rows
    new_df = pd.DataFrame(all_rows)

    # Calculate and add the total pressure exerted by simulated solar wind
    new_df["total_pressure_sim"] = new_df["n_sim"] * k * new_df["T_sim"] + (new_df["B_sim"] ** 2) / (8 * np.pi)

    return new_df

In [6]:

def residual_wind(observed_wind, simulated_wind, donki_data):
    """
    Calculate the residual solar wind parameters between observed and simulated data.

    Parameters:
    observed_wind (pandas DataFrame): 
        A DataFrame containing observed solar wind parameters; B_ob, V_ob, n_ob, T_ob, and total_pressure_ob.

    simulated_wind (pandas DataFrame):
        A DataFrame containing simulated solar wind parameters; B_sim, V_sim, n_sim, T_sim, and total_pressure_sim.

    donki (pandas DataFrame):
        A DataFrame containing the prediction error (PE) for each CME event.

    Returns:
    cme_data (pandas DataFrame):
        A DataFrame containing the residual solar wind parameters; B_diff, V_diff, n_diff, T_diff, and total_pressure_diff.
    """
    # Calculate the residuals, and put them in a new DataFrame
    differences = pd.DataFrame({
    "time_21_5": observed_wind["time_21_5"],
    "B_diff": np.abs(observed_wind["B_ob"] - simulated_wind["B_sim"]),
    "V_diff": np.abs(observed_wind["V_ob"] - simulated_wind["V_sim"]),
    "n_diff": np.abs(observed_wind["n_ob"] - simulated_wind["n_sim"]),
    "T_diff": np.abs(observed_wind["T_ob"] - simulated_wind["T_sim"]),
    "P_diff": np.abs(observed_wind["total_pressure_ob"] - simulated_wind["total_pressure_sim"])
    })
    
    # Now I want to take mean by time_21_5 values and add PE column
    cme_data = differences.groupby("time_21_5", as_index=False).mean()

    # add the columns donki["lat", "long", "speed", "half_width", "cme_transit_obs", "cme_transit_sim"] to the cme_data
    cme_data = cme_data.merge(donki_data[["time_21_5", "lat", "long", "speed", "half_width", "PE", "cme_transit_obs", "cme_transit_sim"]], on="time_21_5")

    # Return the resulting DataFrame
    return cme_data

In [7]:
def normalize_data(data, columns_to_exclude):
    """
    This function takes the data set and returns it after normalizing it,
    excluding specified columns.
    
    Params:
        data (pd.DataFrame): The original DataFrame.
        columns_to_exclude (list): List of column names to exclude from normalization.
    
    Returns:
        pd.DataFrame: The DataFrame with normalized values, excluding the specified columns.
    """
    scaler = MinMaxScaler()
    
    # Exclude the columns that you don't want to normalize
    data_to_normalize = data.drop(columns=columns_to_exclude)
    
    # Fit the scaler on the data to normalize and transform it
    normalized_data = pd.DataFrame(scaler.fit_transform(data_to_normalize),
                                   columns=data_to_normalize.columns,
                                   index=data.index)
    
    # Combine the excluded columns back into the DataFrame
    data_normalized = pd.concat([data[columns_to_exclude], normalized_data], axis=1)
    
    return data_normalized

In [8]:
# the knn function:
def knn_neighbors(train_features, train_target, test_features):
    """
    Perform K-Nearest Neighbors regression with hyperparameter optimization using grid search.
    
    The function performs a grid search over a pre-defined range of hyperparameters to find the optimal K-Nearest Neighbors
    regressor model. It then predicts target values for the provided test feature data using the best found model.
    """
    # Hyperparameter optimization:
    param_grid = {
        'n_neighbors': range(1, 30),
        'weights': ['uniform', 'distance'],
        'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
    }

    # Fit the KNN model to make predictions:
    knn_model = KNeighborsRegressor()
    grid_search = GridSearchCV(knn_model, param_grid, cv=5)
    grid_search.fit(train_features, train_target)
    knn_best = grid_search.best_estimator_
    
    # Make the predictions:
    predictions = knn_best.predict(test_features)
    
    # Optionally, you can return the best estimator and its parameters along with the predictions
    return predictions, knn_best

# the Support vector machine function:
def svm_regression(train_features, train_target, test_features):
    """
    Perform Support Vector Machine regression with hyperparameter optimization using grid search.
    
    The function performs a grid search over a pre-defined range of hyperparameters to find the optimal Support Vector
    Regression model. It then predicts target values for the provided test feature data using the best found model.
    """
    # Hyperparameter optimization:
    param_grid = {
        'C': [0.1, 1, 10, 100, 1000], 
        'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
        'kernel': ['linear', 'rbf']
    }

    # Fit the SVM model to make predictions:
    svm_model = SVR()
    grid_search = GridSearchCV(svm_model, param_grid, cv=5)
    grid_search.fit(train_features, train_target)
    svm_best = grid_search.best_estimator_
    
    # Make the predictions:
    predictions = svm_best.predict(test_features)
    
    return predictions, svm_best

# the linear regression function:
def linear_regression(train_features, train_target, test_features):
    """
    Perform Linear Regression to predict target values based on input features.
    
    The function fits a Linear Regression model using the provided training data and predicts target values for the provided test feature data.
    """
    # Fit the Linear Regression model:
    linear_model = LinearRegression()
    linear_model.fit(train_features, train_target)
    
    # Make the predictions:
    predictions = linear_model.predict(test_features)
    
    return predictions, linear_model

## Workspace

In [9]:
# get the DONKI data cleaned:
filepath = "DONKI_new.xlsx"
donki = clean_donki(filepath)

# Get the OMNI data cleaned:
omni = clean_omni_data("OMNI_COHO1HR_MERGED_MAG_PLASMA_137596.txt")

# process the observed and simulated SW parameters:
processed_wind_obs = process_observed_parameters(omni, donki)
processed_wind_sim = process_simulated_parameters(donki, "ENLIL_data")

# call residual wind to get the final data :)
cme_data = residual_wind(processed_wind_obs, processed_wind_sim, donki)
cme_data = normalize_data(cme_data, ["PE", "time_21_5", "cme_transit_obs", "cme_transit_sim"])

In [10]:
cme_data

,PE,time_21_5,cme_transit_obs,cme_transit_sim,B_diff,V_diff,n_diff,T_diff,P_diff,lat,long,speed,half_width
0,-6.2,2012-03-13 18:55:00,41.666667,35.416667,0.022057,0.715278,0.812868,0.057233,0.006491,0.795455,0.870748,1.000000,0.647887
1,-7.1,2012-07-12 19:35:00,45.850000,38.750000,0.059022,0.130193,0.089135,0.029083,0.008809,0.363636,0.619048,0.538835,0.718310
2,1.2,2012-08-04 18:46:00,74.533333,75.700000,0.097985,0.053302,0.234972,0.087069,0.054837,0.306818,0.387755,0.269417,0.436620
3,-16.0,2012-08-31 22:43:00,60.666667,44.633333,0.347286,0.000966,0.243830,0.000000,0.129802,0.340909,0.149660,0.634951,0.859155
4,-23.4,2012-09-28 01:56:00,68.266667,44.850000,0.048622,0.089849,0.087399,0.010735,0.004689,0.568182,0.782313,0.470874,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,-23.6,2023-02-17 22:08:00,59.733333,36.116667,0.068624,0.215425,0.041094,0.057269,0.029148,0.556818,0.442177,0.629612,0.436620
118,9.9,2023-02-21 11:33:00,72.583333,82.450000,0.231307,0.304295,0.106243,0.405416,0.216288,0.670455,0.741497,0.132524,0.112676
119,13.3,2023-02-25 23:23:00,34.866667,48.216667,0.189282,0.205137,0.478782,1.000000,0.444807,0.625000,0.904762,0.354369,0.619718
120,4.7,2023-03-11 00:44:00,75.233333,79.966667,0.212956,0.051024,0.102078,0.067695,0.101677,0.170455,0.836735,0.157767,0.450704


## Testing out different kinds of cross-validations

In [11]:
# this is gonna be train test split:
X = cme_data[["B_diff"]] # the features 
y = cme_data["PE"] # the target

# split the data:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

# Now get the ML results:
pred = linear_regression(X_train, y_train, X_test)[0]
# convert to series
pred_series = pd.Series(pred, index=y_test.index)

# print the corrected and the test set values:
res = y_test - pred_series
print("tts corrected: ", np.mean(np.abs(res)))
print("tts test_set: ", np.mean(np.abs(y_test)))

tts corrected:  12.97331683554847
tts test_set:  13.635135135135133


In [12]:
from sklearn.model_selection import LeaveOneOut

# Define features and target
X = cme_data[["B_diff", "P_diff", "speed"]]
y = cme_data["PE"]

# Initialize LOOCV
loo = LeaveOneOut()
predictions = []
true_values = []

# Perform LOOCV
count = 0
for train_index, test_index in loo.split(X):
    print("At event: ", count)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Train and predict
    pred = svm_regression(X_train, y_train, X_test)[0]  # Predict single test point
    predictions.append(pred)
    true_values.append(y_test.values[0])
    count += 1

# Convert predictions to Series
pred_series = pd.Series(predictions, index=y.index)

# Calculate error metrics
res = y - pred_series
print("LOOCV corrected: ", np.mean(np.abs(res)))
print("LOOCV test_set: ", np.mean(np.abs(y)))

At event:  0
At event:  1
At event:  2
At event:  3
At event:  4
At event:  5
At event:  6
At event:  7
At event:  8
At event:  9
At event:  10
At event:  11
At event:  12
At event:  13
At event:  14
At event:  15
At event:  16
At event:  17
At event:  18
At event:  19
At event:  20
At event:  21
At event:  22
At event:  23
At event:  24
At event:  25
At event:  26
At event:  27
At event:  28
At event:  29
At event:  30
At event:  31
At event:  32
At event:  33
At event:  34
At event:  35
At event:  36
At event:  37
At event:  38
At event:  39
At event:  40
At event:  41
At event:  42
At event:  43
At event:  44
At event:  45
At event:  46
At event:  47
At event:  48
At event:  49
At event:  50
At event:  51
At event:  52
At event:  53
At event:  54
At event:  55
At event:  56
At event:  57
At event:  58
At event:  59
At event:  60
At event:  61
At event:  62
At event:  63
At event:  64
At event:  65
At event:  66
At event:  67
At event:  68
At event:  69
At event:  70
At event:  71
At

In [ ]:
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd

# Define features and target
X = cme_data[["P_diff"]]
y = cme_data["PE"]

# Initialize K-Fold Cross-Validation
k = 5  # Number of folds
kf = KFold(n_splits=k, shuffle=True, random_state=42)

predictions = []
true_values = []

# Perform K-Fold Cross-Validation
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Train and predict
    pred = svm_regression(X_train, y_train, X_test)[0]
    predictions.extend(pred)      # Append predictions
    true_values.extend(y_test)    # Append actual values

# Convert predictions to Series
pred_series = pd.Series(predictions, index=y.index[:len(predictions)])

# Calculate error metrics
res = pd.Series(true_values) - pred_series
print(f"{k}-Fold corrected: ", np.mean(np.abs(res)))
print(f"{k}-Fold test_set: ", np.mean(np.abs(true_values)))